<a href="https://colab.research.google.com/github/universeatmyfeet/NLP/blob/master/Classwork/sklearn_pipeline_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
import regex as re
import nltk
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from string import digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer
def min_max(X):
  diff = X - np.min(X)
  range1 = np.max(X)- np.min(X)
  final = diff/range1
  return final 

transformer = FunctionTransformer(min_max)
X = [[1,3],[0,1]]
transformer.transform(X)
# use fit's mean and deviation to get inverse transform

array([[0.33333333, 1.        ],
       [0.        , 0.33333333]])

In [0]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
abc = urlopen(link)
zf = ZipFile(io.BytesIO(abc.read()))

In [0]:
spam = pd.read_csv(zf.open('SMSSpamCollection'),sep = '\t',names=["type", "msg"])

In [7]:
spam.head()

,type,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
def toktoktok(text):
  
  text= text.lower()
  text = text.translate(text.maketrans('', '', string.digits))
  text = text.translate(text.maketrans('', '', '''!()-[]{};:'"\,<>./?@#$%^&*_~'''))
  t = word_tokenize(text)
  tokens = ' '.join([word for word in t if word not in stopwords])#removing stop words
  
  return tokens

In [0]:
spam['processed'] = spam["msg"].apply(toktoktok)

In [10]:
spam['processed']

0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry wkly comp win fa cup final tkts st ...
3                     u dun say early hor u c already say
4             nah dont think goes usf lives around though
                              ...                        
5567    nd time tried contact u u £ pound prize claim ...
5568                          ü b going esplanade fr home
5569                          pity mood soany suggestions
5570    guy bitching acted like id interested buying s...
5571                                       rofl true name
Name: processed, Length: 5572, dtype: object

In [0]:
# defining a sklearn pipeline
seq = [('feature_extract', CountVectorizer()),('ensemble', RandomForestClassifier())]
ppl = Pipeline(seq) 
params = {'feature_extract__stop_words':['english'],
          'ensemble__max_depth': [10, 20],
          'ensemble__min_samples_leaf': [1, 2],
          'ensemble__min_samples_split': [2, 5],
          'ensemble__n_estimators': range(10,50,20)}

In [0]:
grid = GridSearchCV(ppl, param_grid=params, cv=5,verbose=1)

In [0]:
X_train,X_test,y_train,y_test = tts(spam['processed'],spam['type'])

In [14]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    8.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feature_extract',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                 

In [15]:
grid.score(X_test,y_test)

0.9260588657573582